## imports

In [1]:
# import libraries
import os
import numpy as np
import torch

## GPU 

In [2]:
gpu_number="4"
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_number

## Paramters

In [3]:
# number of IVIM parameter sets for b-value optimization & training DNN
num_samples = 1000000        
num_samples_val = 10000

# IVIM parameter ranges for the human brain
fmin=0;     fmax=0.2;
Dmin=0;     Dmax=1.8e-3;
Dpmin=0;    Dpmax=20e-3;
S0min=200;    S0max=1700;

# set noise level
S0 = 300
SNR = 100
noise_std = S0/SNR

# number of trials to avoid local minima
num_trials=5 

# number of b-value 
num_bvalue=4

# set file path
file_path = os.getcwd()+'/'

## Generate IVIM parameter sets

In [4]:
# Generate IVIM parameter sets for train & validation
Dp_train = np.zeros(num_samples)
Dt_train = np.zeros(num_samples)
Fp_train = np.zeros(num_samples)
S0_train = np.zeros(num_samples)

Dp_validation = np.zeros(num_samples_val)
Dt_validation = np.zeros(num_samples_val)
Fp_validation = np.zeros(num_samples_val)
S0_validation = np.zeros(num_samples_val)

for i in range(num_samples):
    Dp_train[i] = np.random.uniform(0, 1)*(Dpmax-Dpmin)+Dpmin
    Dt_train[i] = np.random.uniform(0, 1)*(Dmax-Dmin)+Dmin
    Fp_train[i] = np.random.uniform(0, 1)*(fmax-fmin)+fmin
    S0_train[i] = np.random.uniform(0, 1)*(S0max-S0min)+S0min
    
for i in range(num_samples_val):
    Dp_validation[i] = np.random.uniform(0, 1)*(Dpmax-Dpmin)+Dpmin
    Dt_validation[i] = np.random.uniform(0, 1)*(Dmax-Dmin)+Dmin
    Fp_validation[i] = np.random.uniform(0, 1)*(fmax-fmin)+fmin
    S0_validation[i] = np.random.uniform(0, 1)*(S0max-S0min)+S0min

inputdata = np.transpose(np.concatenate(([Fp_train], [Dt_train], [Dp_train], [S0_train]), axis=0))
inputdata_val = np.transpose(np.concatenate(([Fp_validation], [Dt_validation], [Dp_validation], [S0_validation]), axis=0))

# noise will be added to the MRI signals 
noise_train=torch.normal(torch.tensor(np.zeros((num_samples,num_bvalue))),std=noise_std).float().cuda()
noise_validation=torch.normal(torch.tensor(np.zeros((num_samples_val,num_bvalue))),std=noise_std).float().cuda()


## Save data

In [5]:
torch.save(noise_train,file_path+'noise_train.pth')
torch.save(noise_validation,file_path+'noise_validation.pth')

torch.save(inputdata,file_path+'inputdata.pth')
torch.save(inputdata_val,file_path+'inputdata_val.pth')